In [1]:
import pandas as pd
import sys
import pickle5 as pickle
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
from configDict import *

/Users/adi/opt/anaconda3/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
data = pd.read_csv("kenya_stunted_data.csv", index_col="Participant_Id")

In [3]:
x = data.drop("Stunted [EUPATH_0035062]", axis=1)
y = data["Stunted [EUPATH_0035062]"]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=48, stratify=y)

In [5]:
random_states = [100, 12, 124, 24, 34, 50, 68, 72, 75, 44]
scoring = "balanced_accuracy"

In [ ]:
for seed in random_states:
    tpot = TPOTClassifier(generations=100, population_size=100, random_state=seed, verbosity=2, 
                      config_dict=classifier_proba_config_dict, scoring = 'balanced_accuracy',
                          log_file='kenya_stuntedPipeline_' + scoring + '_' + str(seed) + '.log')
    tpot.fit(x_train, y_train)
    tpot.export('kneya_stuntedPipeline_' + scoring + '_' + str(seed) + '.py')
    scores = []
    scores.append([tpot._optimized_pipeline_score, tpot.score(x_test, y_test)])
    scoreDf = pd.DataFrame(scores, columns = ['Training Score', 'Testing Score'])
    scoreDf.to_csv('kenya_stuntedPipeline_' + scoring + '_' +  str(seed) + '.csv')
    bestPipeline = tpot.fitted_pipeline_
    pickle.dump(bestPipeline, open('kenya_stuntedPipeline_' + scoring + '_' + str(seed) + '.pkl', 'wb'))